In [ ]:
from scipy.stats import norm
import numpy as np
import yfinance as yf
import pandas as pd

from datetime import datetime


def GetData(stk, start_date, end_date):
    df = yf.download(stk, start_date, end_date, interval='1d').dropna()
    return (df)


def AddTime(df):
    df['start']=df.index
    df['end']=df.index
    df['dow'] = df.index.dayofweek
    #df['moy']
    df['woy'] = df.index.isocalendar().week + df.index.isocalendar().year * 100
    df['moy']=df.index.strftime('%m-%Y')
    return (df)

def GroupW(df,div,type):
    #df=df[df['dow']!=3]
    #df=df[df['dow']!=2]
    #df=df[df['dow']!=1]
    #df=df[df['dow']!=0]
    if type=="week":
        df['woy']=(df.woy/div).apply(np.ceil)
        df = df.groupby(['woy']).agg({'start':'first','end':'last','Open': 'first', 'Close': 'last'})
        df['days']= (df['end'] - df['start']).dt.days+1
        df=df.set_index('start')
    #if type=="month":
    #    df['moy'] = (df.moy/div).apply(np.ceil)
    #    df = df.groupby(['moy']).agg({'Open': 'first', 'Close': 'last'})
    return (df)

def Outs(df,yield_perc):
    df['K'] = df.Open.round(0)
    df['premium']=df.K*yield_perc
    df['Change']=abs(df.Close-df.K)
    df['PL'] = df.premium-df.Change
    df['LogR'] = np.log((df.Open + df.PL) / df.Open)
    df['OptionYield']=df.premium/df.K
    return(df)

df0=pd.read_csv('option_data_2022-08-19_2022-08-06.csv')


start_date='2012-01-01'
end_date='2022-08-06'
div=2
type="week"
arr=[]
for i in range (0,len(df0.symbol)):
    stock=df0.symbol[i]
    yield_perc=(df0.atm_call_last[i]+df0.atm_put_last[i])/df0.strike[i]
    df = GetData(stock, start_date, end_date)
    df = AddTime(df)
    df=GroupW(df,div,type)
    df=Outs(df,yield_perc)
    log_r=np.sum(df.LogR)
    pos_log=np.sum(df[df.LogR > 0].LogR)
    neg_log=np.sum(df[df.LogR <= 0].LogR)
    pos_periods=len(df[df.LogR > 0].LogR)
    ttl_periods=(len(df.LogR))
    avg_yield=np.average(df.LogR.dropna())
    opt_yield=np.average(df.OptionYield)
    arr.append([stock,yield_perc,log_r,pos_log,neg_log,pos_periods,ttl_periods,avg_yield,opt_yield])
    #print(df)
    #filename=(stock+", "+start_date+", "+end_date+", "+str(div)+".csv")
    #df.to_csv(filename)
    #Sum Log Returns
    #print(np.sum(df.LogR))
    #Sum Positive Log
    #print(np.sum(df[df.LogR > 0].LogR))
    #Sum Negative Log
    #print(np.sum(df[df.LogR <= 0].LogR))
    #Count of Positive Periods
    #print(len(df[df.LogR > 0].LogR))
    #Total Count of Periods
    #print(len(df.LogR))
    #Average Option Yield
    #print(np.average(df.OptionYield))
    #print((np.sum(df[df.LogR > 0].LogR))/(-(np.sum(df[df.LogR <= 0].LogR))))
    #Days/Period
finaldf=pd.DataFrame(data=arr,columns=['stock','IV','log_r','pos_log','neg_log','pos_periods','ttl_periods','avg_yield','opt_yield_perc'])
finaldf['sharpe']=abs(finaldf.pos_log/finaldf.neg_log)
finaldf['winr']=finaldf.pos_periods/finaldf.ttl_periods
finaldf.to_csv('big.csv')
print(finaldf)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed